## Prevent Information Leakage: 

A typical ML downstream process involves input features predicting the output targets. Also, a test set is usually left out to do the evaluation. We make sure to prevent information leakage during evaluation in two aspects:
<ul>
  <li><strong>Testset to trainset</strong>: Only the trainset should be used to train the imputation model.</li>
  <li><strong>Targets to predictors</strong>: To prevent data leakage from target features into predictive features, we have two options:</li>
      <ol>
        <li>Evaluate the imputation and downstream task as a package</li>
        <li>Impute input and output columns of the dataset independently [as if they are separate datasets].</li>
      </ol>
      We use the second method in this notebook to focus our evaluate on the imputation peice only. Furthermore, if we need to seal outputs from each other we need to have 3 datasets in this case. Clearly, by having one column in a dataset, we cannot have a better result than just "mean"-imputation from EDDI. Therefore, at this point we only focus on <strong>input-imputation</strong>. On later notebooks, we will see that the temporal signals can help us with this issue.
</ul>

In [1]:
# pip install azure-storage-blob

In [1]:
import sys
sys.path.append('.')

from helpers.dataprep_utils import compute_missing_ratio, get_variables_metadata
from helpers.sas_utils import get_data_link, get_placeholder_link
from helpers.service_utils import RestEDDI, train_eddi, batchinference_eddi

import pandas as pd
import os
import toml

Load data:

In [2]:
df_sensors = pd.read_csv('./data_generated/sensor_wide.csv', index_col=0)
df_sensors.head(1)

,IN1,IN2,IN3,IN4,IN5,IN6,IN7,IN8,Out1,Out2
time,,,,,,,,,,
0,0.077744,0.795565,-0.665503,0.879321,0.134419,-1.133765,0.253945,0.109987,-0.122686,0.123661


Load EDDI & Storage setup

In [3]:
config = toml.load("./config/config.toml")

container_sas_link = config['blob']['container_sas_link']
subscription_key = config['eddi']['subscription_key']
rest_eddi = RestEDDI(subscription_key, api_version="v2.3")
blob_storage_root = 'sensor_datasets' # name for the blob storage directory within the provided container to store the data

Keep train and test datasets separate; assuming that on the downstream task we will take the first 0.7 part of the data as train and the rest as the test part; we will do the same here:

In [4]:
splitpoint = df_sensors.index[int(df_sensors.shape[0] * 0.7)]
df_train = df_sensors[df_sensors.index < splitpoint]
df_test = df_sensors[df_sensors.index >= splitpoint]

if 'train' not in os.listdir('./data_prepared/'):
    os.mkdir('./data_prepared/train/')

if 'test' not in os.listdir('./data_prepared/'):
    os.mkdir('./data_prepared/test/')

Impute input and output columns of the dataset independently [as if they are separate datasets]. Since we do not have an added value on the output, we skip that for now until future steps.

Get variable metadata

In [5]:
# ideally we should have the metadata provided by the expert so we do not need to rely on current data distribution/min-max to infer the metadata; but we ignore this for now
in_columns = ["IN1", "IN2", "IN3", "IN4", "IN5"]
variables_metadata_in = get_variables_metadata(df_sensors, in_columns,'./config/variables_metadata_in.json')

In [6]:
df_train_in = df_train[in_columns].copy().fillna('')
df_test_in = df_test[in_columns].copy().fillna('')

df_train_in.to_csv('./data_prepared/train/sensor_wide_in.csv', index=False, header=False)
df_test_in.to_csv('./data_prepared/test/sensor_wide_in.csv', index=False, header=False)

Train EDDI imputation model

In [7]:
# uploads the train data to blob storage & returns the SAS link
training_data_source = get_data_link('./data_prepared/train/sensor_wide_in.csv', container_sas_link, directory_name=blob_storage_root)

In [8]:
train_input = {
    "training_data_source": training_data_source,
    "model_hyperparams":{
        "decoder_variances": 1e-6
    },
    "variables_metadata": variables_metadata_in,
    "training_hyperparams":{
        "epochs": 500,
        "iterations":40
    }
}

In [9]:
model_id = train_eddi(train_input, rest_eddi)

<Response [200]>
200
'ab0bf8c5ab26453fa02704418405c516'
Operation status: Completed
train running time: 344.56353759765625 seconds
{'created_time': '2022-04-07T00:51:26.737341+00:00',
 'description': 'Model for efficient decision making with heterogenous data',
 'id': 'ab0bf8c5ab26453fa02704418405c516'}
https://ms-azua-api.azurewebsites.net/saas-api/models/ab0bf8c5ab26453fa02704418405c516?api-version=v2.3


Batch inferencing on train & test dataset and store the results

In [10]:
batch_inference_input = {
    "hyperparameters":
    {
        "sample_count": 50
    }
}

In [11]:
# batch-inference on train-dataset
batch_inference_input["data_source"] = training_data_source
batch_inference_input["output"] = get_placeholder_link('sensor_wide_in_impute.csv', container_sas_link, directory_name=blob_storage_root, delete_prev=True)
batchinference_eddi(batch_inference_input, model_id, rest_eddi)

# download and replace the train data with the missing data
df_train_in_impute = pd.read_csv(batch_inference_input["data_source"], names=in_columns)
df_train_in_impute.to_csv('./data_prepared/train/sensor_wide_in.csv', header=True)

<Response [200]>
200
'aa6e29eba3c44d458e7d88f0acfd604d'
Operation status: Completed
batchinference running time: 93.98148465156555 seconds


In [12]:
# batch-inference on train-dataset
batch_inference_input["data_source"] = get_data_link('./data_prepared/test/sensor_wide_in.csv', container_sas_link, directory_name=blob_storage_root)
batch_inference_input["output"] = get_placeholder_link('sensor_wide_in_impute.csv', container_sas_link, directory_name=blob_storage_root, delete_prev=True)
batchinference_eddi(batch_inference_input, model_id, rest_eddi)

# download and replace the train data with the missing data
df_test_in_impute = pd.read_csv(batch_inference_input["data_source"], names=in_columns)
df_test_in_impute.to_csv('./data_prepared/test/sensor_wide_in.csv', header=True)

<Response [200]>
200
'a679c64eea22447a8e4f147952020d5c'
Operation status: Completed
batchinference running time: 105.16073083877563 seconds


Evaluation